In [ ]:
path = "../data/data186231"
! ls -l $path

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
train = pd.read_csv(f"{path}/train.csv")
train["time"] = pd.to_datetime(train["time"])
train

In [ ]:
COLS = ["time1", "time2", "dv1", "dv2", "mv1", "cv1", "cv2"]

train1 = train.copy()
train1["time1"] = train1["time2"] = train1["time"]
train1 = train1[COLS]
train1

In [ ]:
train2 = [train1]

for n in tqdm(range(1, len(train)+1)):
    _train2 = train.head(n).copy()
    _train2["k"] = 1
    _train2 = _train2.groupby(by="k").agg({
        "time": ["max", "min"],
        "dv1": "mean", 
        "dv2": "mean", 
        "mv1": "mean", 
        "cv1": "mean", 
        "cv2": "mean",
    })
    _train2.columns = [
        f"{icol1}{icol2}".replace("mean", "").replace("max", "1").replace("min", "2")
        for icol1, icol2 in _train2.columns]
    train2.append(_train2)

train2 = pd.concat(train2)
train2

In [ ]:
train2["itime1"] = train2["time1"].astype('int64')//1e9
train2["itime2"] = train2["time2"].astype('int64')//1e9

In [ ]:
testa = pd.read_csv(f"{path}/predict.csv")
testa

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

result = pd.DataFrame()
for T in ["cv1", "cv2"]:
    x, y = train2[["dv1", "dv2", "mv1"]], train2[T]

    def stdError_func(y_test, y):
        return np.sqrt(np.mean((y_test - y) ** 2))

    def R2_1_func(y_test, y):
        return 1 - ((y_test - y) ** 2).sum() / ((y.mean() - y) ** 2).sum()

    def R2_2_func(y_test, y):
        y_mean = np.array(y)
        y_mean[:] = y.mean()
        return 1 - stdError_func(y_test, y) / stdError_func(y_mean, y)


    D = 12
    poly_reg = PolynomialFeatures(degree=D-1)
    X_ploy = poly_reg.fit_transform(x)
    lin_reg_2 = linear_model.LinearRegression()
    lin_reg_2.fit(X_ploy,y)
    predict_y = lin_reg_2.predict(X_ploy)
    strError = stdError_func(predict_y, y)
    R2_1 = R2_1_func(predict_y, y)
    R2_2 = R2_2_func(predict_y, y)
    score = lin_reg_2.score(X_ploy, y)

    print(T)
    print("coefficients", lin_reg_2.coef_[:5])
    print("intercept", lin_reg_2.intercept_)
    print('degree={}: strError={:.2f}, R2_1={:.2f},  R2_2={:.2f}, clf.score={:.2f}'.format(D, strError, R2_1, R2_2, score))
    testa[f"_{T}"] = lin_reg_2.predict(poly_reg.transform(testa[["dv1", "dv2", "mv1"]]))

    result[T] = [_2 if pd.isna(_1) else _1 for _1, _2 in zip(testa[T], testa[f"_{T}"])]


In [ ]:
result.to_csv("submit.csv", index=None)